In [20]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor

from fastapi import FastAPI, UploadFile, File
from io import BytesIO

import pandas as pd
import joblib

In [21]:
df = pd.read_csv('Laptop_price.csv')
X = df.drop(columns=['Price'])
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object']).columns.tolist()
num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([    
        ('imputer', SimpleImputer(strategy='most_frequent')), 
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])
pipeline = Pipeline([
    ('preprocessor', preprocessor), 
   ('model', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5))
])
pipeline.fit(X_train, y_train)
joblib.dump(pipeline, 'laptop_price_model.pkl')

['laptop_price_model.pkl']

In [23]:
!git init 

Reinitialized existing Git repository in C:/Users/amir_/OneDrive/Desktop/LR 3/.git/


In [24]:
!git add pipline.ipynb
!git add Laptop_price.csv

In [25]:
!git commit -m "Добавлен ML-пайплайн"

[master 12e2c92] Добавлен ML-пайплайн
 1 file changed, 39 insertions(+), 49 deletions(-)


In [26]:
!git remote add origin https://github.com/Antelku/lab_pipline.git

error: remote origin already exists.


In [27]:
!git push -u origin master

branch 'master' set up to track 'origin/master'.


To https://github.com/Antelku/lab_pipline.git
   962470a..12e2c92  master -> master


In [28]:
%%writefile app.py

app = FastAPI()

# Загрузка обученной модели
model_path = "laptop_price_model.pkl"
model = joblib.load(model_path)

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    content = await file.read()
    df = pd.read_csv(BytesIO(content))
    predictions = model.predict(df)
    return {"predictions": predictions.tolist()}

Overwriting app.py


In [ ]:
# !nohup uvicorn app:app --host 0.0.0.0 --port 8000 --reload > fastapi.log 2>&1 &
# ngrok не работает в России, не получается зарегистрироваться